<a href="https://colab.research.google.com/github/SidG1204/Re-Identification-in-Single-Feed/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip install ultralytics
!pip install deep_sort_realtime
!pip install opencv-python-headless

In [17]:
from google.colab import files
uploaded = files.upload()

Saving 15sec_input_720p.mp4 to 15sec_input_720p (1).mp4
Saving best.pt to best (1).pt


In [18]:
# Import packages
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
import cv2
import os

In [19]:
# Load model and video
model = YOLO("best.pt")

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30, n_init=3, max_cosine_distance=0.4)

# Load video
cap = cv2.VideoCapture("15sec_input_720p.mp4")

# Video writer setup
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
avi_output = 'output.avi'
out = cv2.VideoWriter(avi_output, cv2.VideoWriter_fourcc(*'XVID'), 30, (frame_width, frame_height))

In [20]:
# Detection + Tracking

while True:
    ret, frame = cap.read()
    if not ret:
        break


    results = model.predict(source=frame, conf=0.3, verbose=False)[0]

    detections = []
    for r in results.boxes:
        cls = int(r.cls[0])
        if cls != 0:
            continue
        x1, y1, x2, y2 = map(int, r.xyxy[0])
        conf = float(r.conf)
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'player'))


    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'Player {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    out.write(frame)

cap.release()
out.release()

In [21]:
# Convert .avi to .mp4
avi_path = 'output.avi'
mp4_path = 'output.mp4'

cap = cv2.VideoCapture(avi_path)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
out_mp4 = cv2.VideoWriter(mp4_path, cv2.VideoWriter_fourcc(*'mp4v'), 30, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break
    out_mp4.write(frame)

cap.release()
out_mp4.release()

In [22]:
# Download the final output
from google.colab import files
files.download('output.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>